# Parte 2: Extracción de tablas de PDF

Primero, instalamos los paquetes e importamos las librerías que necesitamos

In [2]:
!pip install ghostscript
!pip install camelot-py[cv]
!pip install excalibur-py
!apt install ghostscript python3-tk
!pip install --upgrade PyPDF2==2.12.1
!pip install pyreadstat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.3/249.3 kB 19.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of camelot-py[cv] to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 3.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public

In [6]:
import camelot as cm  # libreria de camelot
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import os
from PyPDF2 import PdfFileReader
import re # expresiones regulares
import pyreadstat

# Tablas

In [8]:
 #Tras subir el archivo, extraremos todas las tablas del PDF correspondiente. Corregimos los espacios con strip_text

 tables = cm.read_pdf("/content/sample_data/G1_ANCASH.pdf", pages='1-10', flavor = "lattice", strip_text='\n')

 tables[0]
  #Dado que lee correctamente todos los cuadros, no especificamos coordenadas
  #Asimismo, dado que los nombres de las instituciones tienen números por naturaleza, no los limpiamos

In [9]:
#Creamos un Dataframe con la primera tabla
df_0 = tables[0].df  

# Creamos una lista que almacenará todos los DataFrames, comenzando con df_0, para poder juntarlos
dfs = [df_0] 

# Iteramos sobre las tablas de tables[1] a tables[9]
for i in range(1, 10):
    df_i = tables[i].df 

    # Eliminamos la primera fila del DataFrame original, pues son los nombres de las columnas
    df_i = df_i[1:]

    # Asignamos los nombres de columnas del df_0 al df_i (aún todas sin nombre)
    df_i.columns = df_0.columns

    # Agregamos el DataFrame al final de la lista
    dfs.append(df_i)



In [10]:

# Concatenamos todos los DataFrames en uno solo
merged_df = pd.concat(dfs, ignore_index=True)

# Imprimimos el DataFrame resultante
merged_df

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,#,Año JEC,Código modular,Código local,Nombre de la IE,DRE,UGEL,Ubigeo,Departamento,Provincia,Distrito,Dirección,Área Geográfica
1,62,2015,0597336,018123,VICENTE GUERRERO PALACIOS,DRE ANCASH,UGEL AIJA,020204,ANCASH,AIJA,LA MERCED,RARAMAPAMPA,URBANA
2,63,2015,0411785,017816,GABINO URIBE ANTUNEZ,DRE ANCASH,UGEL AIJA,020201,ANCASH,AIJA,AIJA,YESO,URBANA
3,65,2015,0411868,018608,JULIO C. TELLO,DRE ANCASH,UGEL ANTONIO RAYMONDI,020304,ANCASH,ANTONIO RAYMONDI,CHINGAS,JIRON JULIO C TELLO S/N,URBANA
4,68,2015,0680991,019165,86344 JAVIER HERAUD PEREZ,DRE ANCASH,UGEL ASUNCION,020402,ANCASH,ASUNCION,ACOCHACA,CALLE PRINCIPAL S/N,URBANA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,181,2017,1316835,041208,86644 SAN MARTIN DE PORRES,DRE ANCASH,UGEL YUNGAY,022004,ANCASH,YUNGAY,MATACOTO,CALLE PLAZA DE ARMAS,URBANA
121,182,2017,1321017,041618,86672,DRE ANCASH,UGEL YUNGAY,022005,ANCASH,YUNGAY,QUILLO,CORACOLLO,RURAL
122,183,2017,0906651,041562,86677 SANTIAGO ANTUNEZ DE MAYOLO,DRE ANCASH,UGEL YUNGAY,022005,ANCASH,YUNGAY,QUILLO,AVENIDA YUNGAY S/N,URBANA
123,184,2017,0793216,686232,86640 SEÑOR DE LOS MILAGROS,DRE ANCASH,UGEL YUNGAY,022006,ANCASH,YUNGAY,RANRAHIRCA,JIRON FELIPE VASQUEZ S/N,URBANA


In [11]:
#Asignamos nombres a las columnas

# Obtener la primera fila como encabezados de columnas
new_header = merged_df.iloc[0] 

# Eliminamos la primera fila del DataFrame conjunto
base_final = merged_df[1:]  

# Establecemos los encabezados de columna con la primera fila
base_final.columns = new_header  

#Imprimimos el resultado final
base_final

,#,Año JEC,Código modular,Código local,Nombre de la IE,DRE,UGEL,Ubigeo,Departamento,Provincia,Distrito,Dirección,Área Geográfica
1,62,2015,0597336,018123,VICENTE GUERRERO PALACIOS,DRE ANCASH,UGEL AIJA,020204,ANCASH,AIJA,LA MERCED,RARAMAPAMPA,URBANA
2,63,2015,0411785,017816,GABINO URIBE ANTUNEZ,DRE ANCASH,UGEL AIJA,020201,ANCASH,AIJA,AIJA,YESO,URBANA
3,65,2015,0411868,018608,JULIO C. TELLO,DRE ANCASH,UGEL ANTONIO RAYMONDI,020304,ANCASH,ANTONIO RAYMONDI,CHINGAS,JIRON JULIO C TELLO S/N,URBANA
4,68,2015,0680991,019165,86344 JAVIER HERAUD PEREZ,DRE ANCASH,UGEL ASUNCION,020402,ANCASH,ASUNCION,ACOCHACA,CALLE PRINCIPAL S/N,URBANA
5,70,2015,0577270,019641,20850 MARISCAL CACERES,DRE ANCASH,UGEL BOLOGNESI,020507,ANCASH,BOLOGNESI,COLQUIOC,AVENIDA ALBERTO DE LAS CASAS S/N,URBANA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,181,2017,1316835,041208,86644 SAN MARTIN DE PORRES,DRE ANCASH,UGEL YUNGAY,022004,ANCASH,YUNGAY,MATACOTO,CALLE PLAZA DE ARMAS,URBANA
121,182,2017,1321017,041618,86672,DRE ANCASH,UGEL YUNGAY,022005,ANCASH,YUNGAY,QUILLO,CORACOLLO,RURAL
122,183,2017,0906651,041562,86677 SANTIAGO ANTUNEZ DE MAYOLO,DRE ANCASH,UGEL YUNGAY,022005,ANCASH,YUNGAY,QUILLO,AVENIDA YUNGAY S/N,URBANA
123,184,2017,0793216,686232,86640 SEÑOR DE LOS MILAGROS,DRE ANCASH,UGEL YUNGAY,022006,ANCASH,YUNGAY,RANRAHIRCA,JIRON FELIPE VASQUEZ S/N,URBANA


In [14]:
#Pasamos el Dataframe a formato dta
base_final.to_stata("jec_data_1.dta")

#Descargamos el archivo
files.download("jec_data_1.dta")

<ipython-input-14-3a1a2ccae246>:1: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    #   ->   _
    Año JEC   ->   A_o_JEC
    Código modular   ->   C_digo_modular
    Código local   ->   C_digo_local
    Nombre de la IE   ->   Nombre_de_la_IE
    Dirección   ->   Direcci_n
    Área Geográfica   ->   _rea_Geogr_fica

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  base_final.to_stata("jec_data_1.dta")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>